# Summary of Subject Corpus

## Imports

In [8]:
import requests
import json
import pandas as pd
import numpy as np

## Parameters

In [9]:
corpus_db_url = "http://corpus-db.org/api"
book = 'The Hound of The Baskervilles'
book_id = 3070.0  # have to know the id at the moment

## Get Metadata

In [19]:
metadata = pd.DataFrame(json.loads(requests.get(corpus_db_url + f"/id/{book_id}").text), index=[0])
metadata = metadata.replace('', np.nan).dropna(axis=1)

display(metadata)

,lcsh,creator,downloads,rights_url,authoryearofdeath,_repo,wp_subjects,gutenberg_issued,identifiers,formats,...,id,gutenberg_bookshelf,_version,title,type,wp_literary_genres,publisher,covers,description,filename
0,"{'Blessing and cursing -- Fiction', 'Holmes, S...","{'author': {'agent_name': 'Doyle, Arthur Conan...",348,http://creativecommons.org/about/pdm,1930,The-Hound-of-the-Baskervilles_3070,"['Gothic_novels', '1902_novels', 'Dartmoor', '...",2002-02-01,"{'gutenberg': '3070', 'wikidata': 'Q45192'}",{'text/plain; charset=us-ascii': 'http://www.g...,...,3070.0,"Bestsellers, American, 1895-1923",0.2.0,The Hound of the Baskervilles,Text,['Detective_fiction'],Project Gutenberg,"[{'cover_type': 'generated', 'image_path': 'co...",The Hound of the Baskervilles is the third of ...,/run/media/jon/SAMSUNG/gitenberg/The-Hound-of-...


## Get Full Text

In [25]:
corpus = json.loads(requests.get(corpus_db_url + f"/id/{book_id}/fulltext").text)

print('Book is {0} characters long.'.format(len(corpus[0]['text'])))

Book is 318546 characters long.
